In [1]:
import pandas as pd

import csv
from itertools import islice

def get_row_col(csv_filename, row, col):
    with open(csv_filename, 'rt') as f:
        return next(islice(csv.reader(f), row, row+1))[col]

def import_flow_data(csv_path):
    print(get_row_col(csv_path,5,1))
    flow_raw = pd.read_csv(csv_path, header=10)
    flow_object = DataGroup(get_row_col(csv_path,5,1), get_row_col(csv_path,5,0), flow_raw)
    return flow_object

# flow_raw = import_flow_data('../Data/Spillimacheen/08NA011_QR_Apr-2-2020_06_01_41PM.csv')
    

In [2]:
class DataGroup:
    def __init__(self, river_gauge_title, river_gauge_id, df_river_flow):
        self.river_gauge_title = river_gauge_title
        self.river_gauge_id = river_gauge_id
        self.df = df_river_flow
        print(self.df.info)
        print(f"Created ")
    
        


In [3]:
# flow_raw = import_flow_data('../Data/Spillimacheen/08NA011_QR_Apr-2-2020_06_01_41PM.csv')
flow_csv_path = '../Data/Spillimacheen/smaller_river_test_set.csv'

flow_df_raw = pd.read_csv(flow_csv_path, header=7, index_col=0)
flow_df_raw.drop(["Parameter "],axis=1,inplace=True)
flow_df_raw.index = pd.to_datetime(flow_df_raw.index)

flow = pd.DataFrame()
flow['flow {m3/s)'] = flow_df_raw['Value (m3/s)'].resample('H').first()
print(get_row_col(flow_csv_path,5,1))


SPILLIMACHEEN RIVER NEAR SPILLIMACHEEN


In [4]:
# snow_pillow_height_path = '../Data/Spillimacheen/snow-pillow-conrad-glacier/DataSetExport-1585850817180.csv'
# df_snow = pd.read_csv(snow_pillow_height_path, header=2)



In [13]:
import os
import pytz

# MTC = pytz.timezone('MTC')
df_snow = pd.DataFrame()
folder_to_loop = "../Data/Spillimacheen/snow-pillow-conrad-glacier"
for root, dirs, files in os.walk(folder_to_loop):
    for filename in files:
        temp=pd.read_csv(folder_to_loop+'/'+filename, header=2, index_col=0)
        temp.index = pd.to_datetime(temp.index)
        temp = temp.iloc[:,0:1]
        print(temp.columns[0])
        if len(df_snow) == 0:
            df_snow = temp
        df_snow = pd.concat([df_snow,temp], axis=1)
        # pd.merge(temp, df_snow, left_index=True)
        
        # print(filename)
df_snow = df_snow[['Value (Celsius)', 'Value (Centimetres)']].dropna()
print(df_snow.head(10))

Value (Centimetres)
Value (Celsius)
                     Value (Celsius)  Value (Centimetres)  Value (Centimetres)
Timestamp (UTC)                                                               
2018-11-01 15:00:00             -6.2                156.3                156.3
2018-11-01 16:00:00             -6.2                156.3                156.3
2018-11-01 17:00:00             -5.9                156.7                156.7
2018-11-01 18:00:00             -5.4                156.5                156.5
2018-11-01 19:00:00             -4.2                156.6                156.6
2018-11-01 20:00:00             -5.0                156.4                156.4
2018-11-01 21:00:00             -4.6                156.4                156.4
2018-11-01 22:00:00             -4.6                156.2                156.2
2018-11-01 23:00:00             -4.7                156.2                156.2
2018-11-02 00:00:00             -4.6                156.1                156.1


In [5]:
import csv
from itertools import islice

import mysql
from mysql.connector import Error
import pandas as pd
from datetime import datetime
import os
import pytz
import re


def create_db_connection():
    try:
        print("trying")
        # TODO Create ENV variables to handle database credentials in a correct way.
        # I think I need to use https://direnv.net/
        # db_host = env(DB_HOST, 'localhost')
        # db_port = env.int(DB_PORT, 3306)
        # db_user = env(DB_USER)
        # db_pass = env(DB_PASS)
        # db_name = env(DB_NAME)
        #
        # conn = pymysql.connect(db_host,db_user,db_pass,db_name, use_unicode=True, charset="utf8")

        connection = mysql.connector.connect(host='162.246.156.171',
                                             database='riverapp',
                                             user='riverapp',
                                             password='riverapp',
                                             auth_plugin='mysql_native_password')

        print("connection attempted")
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            cursor.close()
            print(f"You're connected to database: {record}")
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)


def get_sensors_in_datagroup(cursor, datagroup):
    sql = f"select sensor_id from data_group where data_group_id='{datagroup}'"

    cursor.execute(sql)
    result = cursor.fetchall()
    return [x[0] for x in result]


def query_sensor(db_connection, sensor, start, end):
    sql = f"SELECT * from {sensor} WHERE time > '{start}' && time < '{end}'"
    # cursor.execute(sql)
    # result = cursor.fetchall()
    result_df = pd.read_sql(sql, db_connection)
    return result_df


# SELECT sensor_id from 2A33P WHERE time > '2020-02-15 19:00:00' && time < '2020-02-19 17:00:00'

def query_database(data_group, start_time, end_time):
    db_connection = create_db_connection()
    cursor = db_connection.cursor()
    sensors = get_sensors_in_datagroup(cursor, data_group)
    result = pd.DataFrame()
    for sensor in sensors:
        temp = query_sensor(db_connection, sensor, start_time, end_time)
        temp.index = temp['time']
        if result.size is 0:
            result = temp
        result = result.merge(temp,how='outer', left_index=True,right_index=True, suffixes=('','_y'))
        result.drop(list(result.filter(regex='_y$')), axis=1, inplace=True)
    return result


def main():
    result = query_database("SPILLIMACHEEN RIVER NEAR SPILLIMACHEEN", '2020-02-15 19:00:00', '2020-02-19 17:00:00')
    print(result)

In [6]:
result = query_database("SPILLIMACHEEN RIVER NEAR SPILLIMACHEEN", '2020-02-15 19:00:00', '2020-02-19 17:00:00')


trying
connection attempted
Connected to MySQL Server version  8.0.19
You're connected to database: ('riverapp',)
